In [1]:
# Formulation:
#
#       min cx
# s.t. Gx <= h, x[B] Binanry
#      Ax == b, x[I] Integer
#     
# max    X1 + 3X2 + 5X3 + 4X4 + 2X5 + 2X6 + X7 + 5X8 + 3X9
# s.t 
#        X1 + X2 +  X3                                    <= 1
#                          X4 + X5 + X6                   <= 1
#                                           X7 + X8 +  X9 <= 1
#        X1              + X4             + X7            <= 1
#           + X2              + X5             + X8       <= 1
#                   X3             + X6             +  X9 <= 1
#
#    Xi (i = 1...9) >= 0





In [7]:
from cvxopt.glpk import ilp
import numpy as np
from cvxopt import matrix
print (help(ilp))
c = matrix([[1,3,5,4,2,2,1,5,3]], tc='d') #coefficient of cost function, 9 variables
print(c)

Help on built-in function ilp in module cvxopt.glpk:

ilp(...)
    Solves a mixed integer linear program using GLPK.
    
    (status, x) = ilp(c, G, h, A, b, I, B)
    
    PURPOSE
    Solves the mixed integer linear programming problem
    
        minimize    c'*x
        subject to  G*x <= h
                    A*x = b
                    x[k] is integer for k in I
                    x[k] is binary for k in B
    
    ARGUMENTS
    c            nx1 dense 'd' matrix with n>=1
    
    G            mxn dense or sparse 'd' matrix with m>=1
    
    h            mx1 dense 'd' matrix
    
    A            pxn dense or sparse 'd' matrix with p>=0
    
    b            px1 dense 'd' matrix
    
    I            set of indices of integer variables
    
    B            set of indices of binary variables
    
    status       if status is 'optimal', 'feasible', or 'undefined',
                 a value of x is returned and the status string 
                 gives the status of x.  Other po

In [8]:
## Input the coefficient (left-hand side) matrix of the constraints 
coeff=np.array([[1,1,1,0,0,0,0,0,0],
                [0,0,0,1,1,1,0,0,0],
                [0,0,0,0,0,0,1,1,1],
                [1,0,0,1,0,0,1,0,0],
                [0,1,0,0,1,0,0,1,0],
                [0,0,1,0,0,1,0,0,1]
                ],dtype=float)
A=matrix(coeff)
print(A)

[ 1.00e+00  1.00e+00  1.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00 ... ]
[ 0.00e+00  0.00e+00  0.00e+00  1.00e+00  1.00e+00  1.00e+00  0.00e+00 ... ]
[ 0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  1.00e+00 ... ]
[ 1.00e+00  0.00e+00  0.00e+00  1.00e+00  0.00e+00  0.00e+00  1.00e+00 ... ]
[ 0.00e+00  1.00e+00  0.00e+00  0.00e+00  1.00e+00  0.00e+00  0.00e+00 ... ]
[ 0.00e+00  0.00e+00  1.00e+00  0.00e+00  0.00e+00  1.00e+00  0.00e+00 ... ]



In [9]:
##Input the right-hand side vector of the constraint

b = matrix([1,1,1,1,1,1], tc='d')
print(b)

## I is the set of decision variables that take integer values
## B is the set of decision variables that take binary values

I=set(range(9))
B=set(range(9))

print(I)
print(B)

[ 1.00e+00]
[ 1.00e+00]
[ 1.00e+00]
[ 1.00e+00]
[ 1.00e+00]
[ 1.00e+00]

{0, 1, 2, 3, 4, 5, 6, 7, 8}
{0, 1, 2, 3, 4, 5, 6, 7, 8}


In [10]:
(status,x)=ilp(-c,A,b,A,b,I,B) # -c is because we are maximizing the total compatibility score between matches

In [11]:
status

'optimal'

In [12]:
print(x)

[ 0.00e+00]
[ 0.00e+00]
[ 1.00e+00]
[ 1.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 1.00e+00]
[ 0.00e+00]



In [13]:
### Relax the integer/binary constraint

from cvxopt import solvers
Coeff_LP = matrix([[1,0,0,1,0,0,-1,0,0,0,0,0,0,0,0], [1,0,0,0,1,0,0,-1,0,0,0,0,0,0,0],[1,0,0,0,0,1,0,0,-1,0,0,0,0,0,0],
            [0,1,0,1,0,0,0,0,0,-1,0,0,0,0,0],[0,1,0,0,1,0,0,0,0,0,-1,0,0,0,0],
            [0,1,0,0,0,1,0,0,0,0,0,-1,0,0,0],[0,0,1,1,0,0,0,0,0,0,0,0,-1,0,0],
            [0,0,1,0,1,0,0,0,0,0,0,0,0,-1,0],[0,0,1,0,0,1,0,0,0,0,0,0,0,0,-1]],tc='d')
print(Coeff_LP)
RHS_LP = matrix([1,1,1,1,1,1,0,0,0,0,0,0,0,0,0],tc='d')

sol = solvers.lp(-c, Coeff_LP, RHS_LP) # We use -c because we are maximizing revenue
print (sol['x'])

[ 1.00e+00  1.00e+00  1.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00 ... ]
[ 0.00e+00  0.00e+00  0.00e+00  1.00e+00  1.00e+00  1.00e+00  0.00e+00 ... ]
[ 0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  1.00e+00 ... ]
[ 1.00e+00  0.00e+00  0.00e+00  1.00e+00  0.00e+00  0.00e+00  1.00e+00 ... ]
[ 0.00e+00  1.00e+00  0.00e+00  0.00e+00  1.00e+00  0.00e+00  0.00e+00 ... ]
[ 0.00e+00  0.00e+00  1.00e+00  0.00e+00  0.00e+00  1.00e+00  0.00e+00 ... ]
[-1.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00 ... ]
[ 0.00e+00 -1.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00 ... ]
[ 0.00e+00  0.00e+00 -1.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00 ... ]
[ 0.00e+00  0.00e+00  0.00e+00 -1.00e+00  0.00e+00  0.00e+00  0.00e+00 ... ]
[ 0.00e+00  0.00e+00  0.00e+00  0.00e+00 -1.00e+00  0.00e+00  0.00e+00 ... ]
[ 0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00 -1.00e+00  0.00e+00 ... ]
[ 0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00 -1.00e+00 ... ]